In [1]:
import tntorch as tn
import torch
import matplotlib.pyplot as plt

In [2]:
tt_tucker = torch.load('../logs/tt_tucker_r80_00032_longshort_flying_eagle.pt')

In [4]:
[core.requires_grad_() for core in tt_tucker.cores]

[tensor([[[ 1.0077e+05,  2.9927e+00, -1.2513e+00,  ..., -1.3361e-17,
           -8.2831e-18, -5.4773e-18],
          [-4.8448e-01,  1.6362e+04,  1.1623e+02,  ...,  1.8787e-16,
            2.2989e-16,  1.8329e-16],
          [-1.6768e-01,  9.4330e+01, -1.3278e+04,  ..., -6.2502e-16,
            3.4494e-16, -4.1234e-17],
          ...,
          [-1.4211e-13,  1.5846e-12,  1.4467e-13,  ..., -5.8764e-11,
           -5.5224e-11,  1.1366e-10],
          [ 1.7053e-13, -5.1936e-13, -1.2076e-13,  ...,  8.5128e-11,
           -3.8273e-10,  1.2300e-11],
          [-1.8190e-12, -8.5639e-13, -9.3122e-13,  ..., -3.2350e-10,
           -4.5473e-10,  5.7045e-11]]], dtype=torch.float64, requires_grad=True),
 tensor([[[-9.9237e-01, -4.6216e-04,  2.4600e-04,  ...,  6.6905e-22,
            3.1459e-22, -5.4155e-24],
          [ 9.5309e-04, -4.7110e-02,  8.3792e-02,  ..., -1.8391e-21,
           -8.7373e-22,  1.9853e-22],
          [-7.0584e-03,  2.0689e-02,  2.1442e-03,  ..., -1.8263e-21,
           -5.25

In [5]:
[U.requires_grad_() for U in tt_tucker.Us]

[tensor([[-0.0648,  0.0727,  0.0537,  ..., -0.0223,  0.0227, -0.0142],
         [-0.0646,  0.0722,  0.0538,  ..., -0.0119,  0.0115, -0.0095],
         [-0.0644,  0.0716,  0.0538,  ..., -0.0034,  0.0027, -0.0047],
         ...,
         [-0.0620,  0.0587, -0.0665,  ..., -0.0074,  0.0161,  0.0196],
         [-0.0622,  0.0592, -0.0665,  ..., -0.0277,  0.0597,  0.0756],
         [-0.0624,  0.0598, -0.0665,  ..., -0.0526,  0.1129,  0.1443]],
        dtype=torch.float64, requires_grad=True),
 tensor([[-4.6175e-02, -1.1925e-02, -5.3339e-02,  ...,  2.4035e-02,
           5.8439e-04,  1.8512e-02],
         [-4.6147e-02, -1.2013e-02, -5.3264e-02,  ...,  7.8441e-03,
           3.1346e-04,  5.4582e-03],
         [-4.6121e-02, -1.2098e-02, -5.3196e-02,  ..., -4.2695e-03,
           7.8225e-05, -4.0708e-03],
         ...,
         [-5.3248e-02,  7.9884e-02, -3.6859e-02,  ..., -2.2047e-03,
          -1.5415e-04, -1.7569e-03],
         [-5.3377e-02,  7.9950e-02, -3.7480e-02,  ..., -1.5115e-02,
       

In [6]:
MIN_TSDF = -0.05
MAX_TSDF = 0.05
NUM_EPOCHS = 100
NUM_SAMPLES = 100000
RES = 512

optimizer = torch.optim.SGD(tt_tucker.Us + tt_tucker.cores, lr=1e-7)

In [7]:
gt_sdf = torch.load(
    '/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/posed/sdf_watertight_longshort_flying_eagle.000001.pt')['sdf']

In [8]:
reconstruction = tt_tucker[..., 0].torch()
loss = torch.sqrt((reconstruction.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF) -
        gt_sdf.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF))**2).mean()
loss

tensor(3.7846e-05, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [9]:
for i in range(NUM_EPOCHS):
    optimizer.zero_grad()
    reconstruction = tt_tucker[..., 0].torch()
    idxs = torch.stack([torch.randint(0, RES, (NUM_SAMPLES, )) for i in range(3)], dim=-1)
    loss = (reconstruction[idxs[:, 0], idxs[:, 1], idxs[:, 2]].clamp_max(MAX_TSDF).clamp_min(MIN_TSDF) -
            gt_sdf[idxs[:, 0], idxs[:, 1], idxs[:, 2]].clamp_max(MAX_TSDF).clamp_min(MIN_TSDF)).abs().mean()
    loss.backward()
    print(f'Epoch: {i}, loss: {loss.item()}')
    optimizer.step()

Epoch: 0, loss: 3.906509957385132e-05
Epoch: 1, loss: 3.646261384705368e-05
Epoch: 2, loss: 3.8270028470228594e-05
Epoch: 3, loss: 3.856698151915819e-05
Epoch: 4, loss: 3.791544967697622e-05
Epoch: 5, loss: 3.763288165714733e-05
Epoch: 6, loss: 3.799892000675463e-05
Epoch: 7, loss: 3.787194595731976e-05
Epoch: 8, loss: 3.807628089589584e-05
Epoch: 9, loss: 3.679512747785073e-05
Epoch: 10, loss: 3.7406843195869396e-05
Epoch: 11, loss: 3.886416371459205e-05
Epoch: 12, loss: 3.789518809199137e-05
Epoch: 13, loss: 3.864938457670692e-05
Epoch: 14, loss: 3.844092546129123e-05
Epoch: 15, loss: 3.773827505114413e-05
Epoch: 16, loss: 3.7813343573530986e-05
Epoch: 17, loss: 3.706643184538261e-05
Epoch: 18, loss: 3.7599995814190985e-05
Epoch: 19, loss: 3.8473274093241365e-05
Epoch: 20, loss: 3.697890643322235e-05
Epoch: 21, loss: 3.773860507369062e-05
Epoch: 22, loss: 3.806127520682001e-05
Epoch: 23, loss: 3.877859385704054e-05
Epoch: 24, loss: 3.762941612304047e-05
Epoch: 25, loss: 4.00943481890

In [8]:
for i in range(NUM_EPOCHS):
    optimizer.zero_grad()
    reconstruction = tt_tucker[..., 0].torch()
    idxs = torch.stack([torch.randint(0, RES, (NUM_SAMPLES, )) for i in range(3)], dim=-1)
    loss = (reconstruction[idxs[:, 0], idxs[:, 1], idxs[:, 2]].clamp_max(MAX_TSDF).clamp_min(MIN_TSDF) -
            gt_sdf[idxs[:, 0], idxs[:, 1], idxs[:, 2]].clamp_max(MAX_TSDF).clamp_min(MIN_TSDF)).abs().mean()
    loss.backward()
    print(f'Epoch: {i}, loss: {loss.item()}')
    optimizer.step()

Epoch: 0, loss: 4.019383251785526e-05
Epoch: 1, loss: 3.583481492037336e-05
Epoch: 2, loss: 3.9116720256715485e-05
Epoch: 3, loss: 3.884746435025234e-05
Epoch: 4, loss: 3.750155832617766e-05
Epoch: 5, loss: 3.774428726790507e-05
Epoch: 6, loss: 3.795775454124457e-05
Epoch: 7, loss: 3.6453055640710346e-05
Epoch: 8, loss: 3.69181813416022e-05
Epoch: 9, loss: 3.790171391379192e-05
Epoch: 10, loss: 3.61141585261772e-05
Epoch: 11, loss: 3.570005030584897e-05
Epoch: 12, loss: 3.877619241930353e-05
Epoch: 13, loss: 3.8484673152759384e-05
Epoch: 14, loss: 3.608358679638437e-05
Epoch: 15, loss: 3.716449964888181e-05
Epoch: 16, loss: 3.8267337701679986e-05
Epoch: 17, loss: 3.880692067317823e-05
Epoch: 18, loss: 3.800318532526119e-05
Epoch: 19, loss: 3.668389794644888e-05
Epoch: 20, loss: 3.855890155137556e-05
Epoch: 21, loss: 3.8275343103724673e-05
Epoch: 22, loss: 3.874159783335278e-05
Epoch: 23, loss: 3.677468815222015e-05
Epoch: 24, loss: 3.69458470457616e-05
Epoch: 25, loss: 3.81065734144781

In [9]:
(reconstruction.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF) -
        gt_sdf.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF)).abs().mean()

tensor(3.6321e-05, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [11]:
torch.sqrt((reconstruction.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF) -
        gt_sdf.clamp_max(MAX_TSDF).clamp_min(MIN_TSDF))**2).mean()

tensor(3.7646e-05, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [4]:
import skimage
import time

coords = torch.tensor(
    torch.load(
        '/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/posed/sdf_watertight_longshort_flying_eagle.000001.pt')['coords'])

In [3]:
reconstruction = tt_tucker[..., 0].torch()

In [6]:
RES = 512
t0 = time.time()
verts, faces, _, _ = skimage.measure.marching_cubes(
    reconstruction.detach().numpy(),
    level=0.,
    spacing=(coords[3:] - coords[:3]) / RES)
verts = torch.tensor(verts.copy())
faces = torch.tensor(faces.copy())
print(f'Marching cube finished. Took: {time.time() - t0} s.')

Marching cube finished. Took: 2.0119144916534424 s.


In [7]:
import trimesh

In [8]:
mesh = trimesh.base.Trimesh(vertices=verts, faces=faces)

obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open('./sgd_optimised.obj', 'w') as f:
    f.write(obj)

In [30]:
mesh

<trimesh.Trimesh(vertices.shape=(217280, 3), faces.shape=(434568, 3))>

In [33]:
torch.save(tt_tucker, '../logs/tt_tucker_sgd_optimised_r80_00032_longshort_flying_eagle.pt')